# Records

The builtin types (`Bit`, `BitVector`, `Unsigned` and so on) have some special properties that set them apart from most 'normal' Python classes. They are assignable in synthesizable contexts and can be type qualified (wrapped in Signal/Variable/Temporary).

To mimic this behavior, user define data types must implement, some member functions and handle the magic `_qualifier_` argument in their constructor (see [assignable types](12_assignable_types.ipynb)). Since this is error prone and results in much duplicated code, the standard library provides `std.Record` to automate all that.

Inheriting from `std.Records` is similar to the `@dataclass` decorator known from Python. It inspects the type annotations of a class and adds member functions.

`std.Record` implements the following methods for derived classes:

* `__init__`

    * Handles the magic `_qualifier_` argument so records can be type qualified. The qualifier is delegated to the record members.
    * When called with no arguments, the members created but not initialized (no default value).
    * A single positional argument of `cohdl.Null` or `cohdl.Full` initializes members to all zeros/ones
    * A single positional argument of the same type copies all members from the argument
    * Members can be individually initialized using keyword arguments

* assignment operators (`__ilshift__`, `.next`, `__ixor__`, `.push`, `__imatmul__` and `.value`)

    * same behavior as builtin types
    * only Signal qualified records can use `<<=` and `^=`
    * only Variable qualified records can use `@=`

* serialization functions (`_to_bits_`, `_from_bits_` and `_count_bits`)

    * used to convert instances of the class to/from BitVectors. (see `std.to_bits`, `std.from_bits` and `std.count_bits`)
    * also needed to produce `std.Arrays` of the record type

* comparison operators (`__eq__` and `__ne__`)

    Two instances are considered equal, when all their members compare equal. Otherwise they are not equal.

* `__str__` and `__repr__`

    pretty print object/members


In [ ]:
from __future__ import annotations

from cohdl import Port, BitVector, Signed, Unsigned, Signal, Bit, Null
from cohdl import Entity

from cohdl import std


class Coord(std.Record):
    x: Signed[16]
    y: Signed[16]

    def __add__(self, other: Coord):
        return Coord(self.x + other.x, self.y + other.y)


class Inner(std.Record):
    a: Bit
    b: BitVector[7:0]


class Outer(std.Record):

    # record can be nested
    inner_1: Inner
    inner_2: Inner

    outer_bit: Bit
    outer_unsigned: Unsigned[7:0]

    # records can contains arrays of other records
    outer_array: std.Array[Inner, 4]


class MyEntity(Entity):
    x_in = Port.input(Signed[16])
    y_in = Port.input(Signed[16])

    x_out = Port.output(Signed[16])
    y_out = Port.output(Signed[16])

    def architecture(self):
        # just like builtin types, instances of the record are constant
        # unless they are type qualified
        coord_const = Coord(1, 7)

        # use TypeQualifiers to produce runtime variable record objects
        outer_signal = Signal[Outer]()

        # std.Ref works like a TypeQualifier
        # here the elements of coord_out are not copied but taken by reference
        coord_out = std.Ref[Coord](self.x_out, self.y_out)

        @std.sequential
        def logic():
            nonlocal coord_out, outer_signal
            outer_signal <<= Null
            coord_in = Coord(self.x_in, self.y_in)

            # coord is not a signal
            # but can be assigned like one
            coord_out <<= coord_in + coord_const


vhdl = std.VhdlCompiler.to_string(MyEntity)
print(vhdl)


## Record templates

Record types can be generic.

In [ ]:
from __future__ import annotations

from cohdl import Port, BitVector, Signed
from cohdl import Entity

from cohdl import std

# obtain generic type and use it to produce a generic record
T = std.TemplateArg.Type

class Coord(std.Record[T]):
    x: T
    y: T

    def __add__(self, other: Coord) -> Coord:
        # use type(self) instead of Coord
        # to get specialized template type
        return type(self)(
            self.x+other.x,
            self.y+other.y
        )

class MyEntity(Entity):
    x_in = Port.input(BitVector[32])
    y_in = Port.input(BitVector[32])

    x_out_signed = Port.output(BitVector[32])
    y_out_signed = Port.output(BitVector[32])

    x_out_unsigned = Port.output(BitVector[16])
    y_out_unsigned = Port.output(BitVector[16])

    def architecture(self):
        # specialize the generic 
        out_signed = std.Ref[Coord[Signed[32]]](self.x_out_signed, self.y_out_signed)

        @std.concurrent
        def logic_signed():
            nonlocal out_signed

            # specialize generic Coord with Signed[32]
            coord_in = Coord[Signed[32]](self.x_in, self.y_in)

            # coord is not a signal
            # but can be assigned like one
            out_signed <<= coord_in
        
        @std.concurrent
        def logic_unsigned():

            # specialize generic Coord with Unsigned[16]
            a = Coord[Unsigned[16]](self.x_in[15:0], self.y_in[15:0])
            b = Coord[Unsigned[16]](self.x_in[31:16], self.y_in[31:16])

            # create a new instance of Coord from constant arguments
            c = Coord[Unsigned[16]](3, 4)

            sum = a + b + c

            self.x_out_unsigned <<= sum.x
            self.y_out_unsigned <<= sum.y

vhdl = std.VhdlCompiler.to_string(MyEntity)
print(vhdl)
